Importar excel

In [1]:
import pandas as pd

# Cargar Excel
df = pd.read_excel("BASEDATOS_COMPLETA.xlsx")

# Opcional: quedarnos solo con textos en inglés
df = df[df["lenguage"] == "en"].copy()


Funcion para recortar fichero

In [2]:
def stratified_sample(df, label_col="label", total_n=10_000, frac_label1=0.58, random_state=42):
    """
    Crea un subconjunto estratificado de tamaño total_n.
    frac_label1 = proporción de la clase 1 (por ejemplo 0.5 -> 50/50; 0.6 -> 60/40).
    """
    n1 = int(total_n * frac_label1)
    n0 = total_n - n1

    df1 = df[df[label_col] == 1]
    df0 = df[df[label_col] == 0]

    # Por si acaso no hay suficientes de una clase:
    n1 = min(n1, len(df1))
    n0 = min(n0, len(df0))
    total_n = n1 + n0  # tamaño real final

    sample1 = df1.sample(n=n1, random_state=random_state)
    sample0 = df0.sample(n=n0, random_state=random_state)

    return pd.concat([sample1, sample0]).sample(frac=1, random_state=random_state).reset_index(drop=True)


Filtrado para descartar textos demasiado cortos

In [3]:
df["text_len"].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

threshold = df["text_len"].quantile(0.5)   # mediana ~2440
df_long = df[df["text_len"] >= threshold].copy()

print(df_long.shape)
print(df_long["label"].value_counts())

df_long = stratified_sample(df_long, total_n=10_000, frac_label1=0.58)
print(df_long["label"].value_counts())

# df_long.to_csv("BASE_FINAL_TRABAJO.csv", index=False)
df_long.to_excel("BASEDATOS_REDUCIDA.xlsx", index=False)


(34491, 11)
label
0    19007
1    15484
Name: count, dtype: int64
label
1    5800
0    4200
Name: count, dtype: int64
